In [34]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
print(talib.__version__)


0.5.1


# test multipe setups on 1 asset 
- download 1 df with OHLC 
- use close for indicator calculation 

In [35]:
# Prepare data
start = '2019-01-01 UTC'  # crypto is in UTC
end = datetime.utcnow()
dir_name = '/Users/andre/Documents/Python/trading_clone_singapore_DO/trading_singapore_digitalocean/vectorBT'
os.makedirs(dir_name, exist_ok=True)
timeframe = '4h'

# Function to download and save data
def download_and_save_data(symbol, file_name, overwrite=False):
    joined = os.path.join(dir_name, file_name)
    if os.path.exists(joined) and not overwrite:
        data = pd.read_pickle(joined)
        print(f'File {file_name} exists already')
    else:
        data = vbt.CCXTData.download(exchange='kucoin', symbols=symbol, timeframe=timeframe, start=start, end=end).get()
        data.to_pickle(joined)
        print(f'Downloaded and saved {symbol} data')
    return data

# List of assets
assets = ['ETH']
divider = '/'  # Divider for the assets
base_value = '/USDT'  # Base value for the assets

# Dictionary to store data
data_dict = {}

# Download data for each asset
for asset in assets:
    symbol = asset.lower()
    file_name = f'{symbol}_price_{timeframe}.pkl'
    data_dict[symbol] = download_and_save_data(asset + base_value, file_name, overwrite=False)

# Align the DataFrames by their index
aligned_data = [data_dict[symbol].align(data_dict[assets[0].lower()], join='inner')[0] for symbol in data_dict]

# Concatenate the aligned DataFrames
comb_price = pd.concat(aligned_data, axis=1, keys=[asset for asset in assets])
comb_price.columns.names = ['symbol', 'attribute']

# Create DataFrames for Open, High, Low, and Close prices
comb_open = pd.concat([data_dict[symbol]['Open'] for symbol in data_dict], axis=1, keys=[f'{symbol}_open' for symbol in data_dict])
comb_high = pd.concat([data_dict[symbol]['High'] for symbol in data_dict], axis=1, keys=[f'{symbol}_high' for symbol in data_dict])
comb_low = pd.concat([data_dict[symbol]['Low'] for symbol in data_dict], axis=1, keys=[f'{symbol}_low' for symbol in data_dict])
comb_close_new = pd.concat([data_dict[symbol]['Close'] for symbol in data_dict], axis=1, keys=[f'{symbol}_close' for symbol in data_dict])

# Display the resulting DataFrames
print("Open Prices:")
print(comb_open.head())

print("\nHigh Prices:")
print(comb_high.head())

print("\nLow Prices:")
print(comb_low.head())

print("\nClose Prices:")
print(comb_close_new.head())


File eth_price_4h.pkl exists already
Open Prices:
                             eth_open
Open time                            
2019-01-01 00:00:00+00:00  131.361407
2019-01-01 04:00:00+00:00  131.352223
2019-01-01 08:00:00+00:00  133.619363
2019-01-01 12:00:00+00:00  133.660560
2019-01-01 16:00:00+00:00  132.956125

High Prices:
                             eth_high
Open time                            
2019-01-01 00:00:00+00:00  132.172291
2019-01-01 04:00:00+00:00  134.623954
2019-01-01 08:00:00+00:00  135.068180
2019-01-01 12:00:00+00:00  135.609804
2019-01-01 16:00:00+00:00  135.463478

Low Prices:
                              eth_low
Open time                            
2019-01-01 00:00:00+00:00  130.082442
2019-01-01 04:00:00+00:00  131.294581
2019-01-01 08:00:00+00:00  133.499608
2019-01-01 12:00:00+00:00  132.050329
2019-01-01 16:00:00+00:00  131.103022

Close Prices:
                            eth_close
Open time                            
2019-01-01 00:00:00+00:00  131.227

## creating indicator to set signals long ONLY for simplicity

In [36]:
# Define a condition
skip_cell = False

# Check the condition and execute the cell only if the condition is False
if not skip_cell:
    # Your code here
    print("This cell is executed.")

    def ma_cross_rsi_long_only(close_price, ma1_fast, ma2_slow, rsi_buy,rsi_window, ma_diff=0.02):
        # Calculate moving averages

        SMA = vbt.IndicatorFactory.from_talib('SMA')
        ma1 = SMA.run(close_price, timeperiod=ma1_fast).real.to_numpy()
        ma2 = SMA.run(close_price, timeperiod=ma2_slow).real.to_numpy()
        # ma1.info()
        # ma2.info()
        # ma1 = vbt.MA.run(close_price, window=ma1_fast).ma.to_numpy()
        # ma2 = vbt.MA.run(close_price, window=ma2_slow).ma.to_numpy()
        
        # Calculate RSI and determine buy signals
        RSI = vbt.IndicatorFactory.from_talib('RSI')
        rsi = RSI.run(close_price, timeperiod=rsi_window).real.to_numpy()
        # rsi.info()

        # rsi = vbt.RSI.run(close_price, window=14).rsi.to_numpy()
        rsi_buy_signal = rsi > rsi_buy
        
        # Generate buy signals
        signal_buy = (close_price > ma1) & rsi_buy_signal & (ma1 > ma2)
        caution_long = (close_price < ma1) & (ma1 > ma2)
        
        # Check if ma1 and ma2 are too close to each other (percentage difference)
        percentage_diff = np.abs(ma1 - ma2) / ma2
        undecided = percentage_diff < ma_diff
        
        # Define conditions and corresponding values for signals
        conditions = [
            signal_buy & ~undecided,
            caution_long & ~undecided,
            undecided
        ]

        values = [
            100,  # Strong buy signal (close price above short trend MA and RSI above the buy threshold)
            50,   # Caution long
            0     # Undecided
        ]

        # Use np.select to apply the conditions and assign the corresponding values
        signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

        return signal

    # Create the indicator factory
    ma_cross_slow_long_only = vbt.IndicatorFactory(
        class_name='MA_Cross_rsi',
        short_name='ma_rsi',
        input_names=['Close'],
        param_names=['ma1_fast', 'ma2_slow', 'rsi_buy', 'rsi_window'],
        output_names=['ma_rsi']
    ).from_apply_func(ma_cross_rsi_long_only, ma1_fast=20, ma2_slow=50, rsi_buy=70, rsi_window=14)

    # # # Define window sizes and RSI thresholds using np.arange
    # ma_trend_fast = np.arange(20, 101, 10)  # Range from 20 to 50 with step 5
    # ma_trend_slow = np.arange(100, 400, 50)  # Range from 50 to 100 with step 10
    # rsi_buy = np.arange(50, 70, 5)  # Range from 60 to 80 with step 5
    # rsi_window = np.arange(5, 25, 2)  # Range from 10 to 20 with step 2
    ma_trend_fast = 50
    ma_trend_slow = 400
    rsi_buy = 70
    rsi_window = [14,7]

    # # Print the shapes of the input arrays
    # print(f"Shape of ma_trend_fast: {ma_trend_fast.shape}")
    # print(f"Shape of ma_trend_slow: {ma_trend_slow.shape}")
    # print(f"Shape of rsi_buy: {rsi_buy.shape}")
    # print(f"Shape of rsi_window: {rsi_window.shape}")

    # Run the indicator
    signal = ma_cross_slow_long_only.run(
        comb_close_new,
        ma1_fast=ma_trend_fast,
        ma2_slow=ma_trend_slow,
        rsi_buy=rsi_buy,
        rsi_window=rsi_window,
        param_product=True  
    )

    signal.ma_rsi
else:
    print("This cell is skipped.")

This cell is executed.


In [37]:
signal.ma_rsi.head()

ma_rsi_ma1_fast            50   
ma_rsi_ma2_slow           400   
ma_rsi_rsi_buy             70   
ma_rsi_rsi_window          14 7 
Open time                       
2019-01-01 00:00:00+00:00   0  0
2019-01-01 04:00:00+00:00   0  0
2019-01-01 08:00:00+00:00   0  0
2019-01-01 12:00:00+00:00   0  0
2019-01-01 16:00:00+00:00   0  0

# caluculation for needed input(from_order_func) of columns to match indicator possibilities
- check how many assets will be tested 
- check how many columns (different setups) will be produces
- devide by aamount of asset to to find the number of copy needed for specific values closing price
- need to be done because from_order funct needs goes through column by column to to calculate the trades 
- that means every indicator entry column need a asset close column to check against

In [38]:
# if not os.path.exists('new_1h_ma_rsi.pkl'):
#     signal.ma_rsi.to_pickle('new_1h_ma_rsi.pkl')
#     df_custom_indiator = signal.ma_rsi
# else:
#     df_custom_indiator = pd.read_pickle('1h_ma_rsi.pkl')

# df_custom_indiator.columns

df_custom_indiator = signal.ma_rsi

print(len(df_custom_indiator.columns)) 
print(len(assets))
needed_copies = int(len(df_custom_indiator.columns)/len(assets))
print(needed_copies)

2
1
2


- copy OHLC df to match indicator 
- split OHLC in seperate df for O H L C


In [39]:
import numpy as np
import pandas as pd

n = int(needed_copies)


symbols = list(data_dict.keys())

# Create DataFrames for Open, High, Low, and Close prices
open_df = pd.DataFrame({symbol: data_dict[symbol]['Open'] for symbol in symbols})
high_df = pd.DataFrame({symbol: data_dict[symbol]['High'] for symbol in symbols})
low_df = pd.DataFrame({symbol: data_dict[symbol]['Low'] for symbol in symbols})
close_df = pd.DataFrame({symbol: data_dict[symbol]['Close'] for symbol in symbols})

# Repeat the data for each symbol n times without loops
comb_open = np.hstack([np.tile(open_df[symbol].values[:, None], n) for symbol in symbols])
comb_high = np.hstack([np.tile(high_df[symbol].values[:, None], n) for symbol in symbols])
comb_low = np.hstack([np.tile(low_df[symbol].values[:, None], n) for symbol in symbols])
comb_close_new1 = np.hstack([np.tile(close_df[symbol].values[:, None], n) for symbol in symbols])

# Generate column names
open_columns = [f'{symbol}_open{copy+1}' for symbol in symbols for copy in range(n)]
high_columns = [f'{symbol}_high{copy+1}' for symbol in symbols for copy in range(n)]
low_columns = [f'{symbol}_low{copy+1}' for symbol in symbols for copy in range(n)]
close_columns = [f'{symbol}_close{copy+1}' for symbol in symbols for copy in range(n)]

# Convert back to DataFrames
comb_open_df = pd.DataFrame(comb_open, columns=open_columns, index=open_df.index)
comb_high_df = pd.DataFrame(comb_high, columns=high_columns, index=high_df.index)
comb_low_df = pd.DataFrame(comb_low, columns=low_columns, index=low_df.index)
comb_close_df = pd.DataFrame(comb_close_new1, columns=close_columns, index=close_df.index)

# Display the resulting DataFrames with n copies
print("Open Prices with n copies:")
print(comb_open_df.head())

print("\nHigh Prices with n copies:")
print(comb_high_df.head())

print("\nLow Prices with n copies:")
print(comb_low_df.head())

print("\nClose Prices with n copies:")
print(comb_close_df.head())

Open Prices with n copies:
                            eth_open1   eth_open2
Open time                                        
2019-01-01 00:00:00+00:00  131.361407  131.361407
2019-01-01 04:00:00+00:00  131.352223  131.352223
2019-01-01 08:00:00+00:00  133.619363  133.619363
2019-01-01 12:00:00+00:00  133.660560  133.660560
2019-01-01 16:00:00+00:00  132.956125  132.956125

High Prices with n copies:
                            eth_high1   eth_high2
Open time                                        
2019-01-01 00:00:00+00:00  132.172291  132.172291
2019-01-01 04:00:00+00:00  134.623954  134.623954
2019-01-01 08:00:00+00:00  135.068180  135.068180
2019-01-01 12:00:00+00:00  135.609804  135.609804
2019-01-01 16:00:00+00:00  135.463478  135.463478

Low Prices with n copies:
                             eth_low1    eth_low2
Open time                                        
2019-01-01 00:00:00+00:00  130.082442  130.082442
2019-01-01 04:00:00+00:00  131.294581  131.294581
2019-01-01 08:00:0

In [40]:
comb_close_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13114 entries, 2019-01-01 00:00:00+00:00 to 2024-12-25 12:00:00+00:00
Freq: 4h
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   eth_close1  13114 non-null  float64
 1   eth_close2  13114 non-null  float64
dtypes: float64(2)
memory usage: 307.4 KB


# copy dataframe that contains assest closing price 

In [41]:
skip = True
if not skip:
    # Your code here
    print("This cell is executed.")

    # Define the number of copies    
    n = int(needed_copies)
    # speed up by converting to numpy

    # Create DataFrames for Open, High, Low, and Close prices with n copies
    comb_open = pd.concat([data_dict[symbol]['Open'].rename(f'{symbol}_open{i+1}') for i in range(n) for symbol in data_dict], axis=1)
    comb_high = pd.concat([data_dict[symbol]['High'].rename(f'{symbol}_high{i+1}') for i in range(n) for symbol in data_dict], axis=1)
    comb_low = pd.concat([data_dict[symbol]['Low'].rename(f'{symbol}_low{i+1}') for i in range(n) for symbol in data_dict], axis=1)
    comb_close_new = pd.concat([data_dict[symbol]['Close'].rename(f'{symbol}_close{i+1}') for i in range(n) for symbol in data_dict], axis=1)

    # Display the resulting DataFrames with n copies
    print("Open Prices with n copies:")
    print(comb_open.head())

    print("\nHigh Prices with n copies:")
    print(comb_high.head())

    print("\nLow Prices with n copies:")
    print(comb_low.head())

    print("\nClose Prices with n copies:")
    print(comb_close_new.head())
else:
    print("This cell is skipped.")

This cell is skipped.


## compare the shape of the copied closing prices 

In [42]:
comb_close_copy = comb_close_df.copy()
df_combined_close = pd.DataFrame(comb_close_copy)

# Assuming df_indi and df_C_price are your DataFrames
# Check if they have the same shape
if df_custom_indiator.shape == df_combined_close.shape:
    # Rename the columns of df_C_price to match the column names of df_indi
    df_combined_close.columns = df_custom_indiator.columns
    print("Columns of df_C_price have been renamed to match df_indi.")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_combined_close.head()

Columns of df_C_price have been renamed to match df_indi.


ma_rsi_ma1_fast                    50            
ma_rsi_ma2_slow                   400            
ma_rsi_rsi_buy                     70            
ma_rsi_rsi_window                  14          7 
Open time                                        
2019-01-01 00:00:00+00:00  131.227012  131.227012
2019-01-01 04:00:00+00:00  133.576936  133.576936
2019-01-01 08:00:00+00:00  133.724401  133.724401
2019-01-01 12:00:00+00:00  132.976895  132.976895
2019-01-01 16:00:00+00:00  135.355827  135.355827

In [49]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if low[c.i,c.col] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly)
            return value

        # sl update for long position

        if c.i >= 5:  # Ensure there are enough bars to look back
            lowest_low = np.min(low[c.i-2:c.i,c.col])  # Find the lowest low in the last 5 bars
            if (entries[c.i, c.col] == 75):
                # print('sl update before',c.i,sl_prices[c.i])
                update1 = lowest_low * 0.9
                if update1 > sl_prices[c.i]:
                    sl_prices[:]= update1
                    # print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 50):  
                update2 = lowest_low * 0.95
                if update2 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update2       
            if (entries[c.i, c.col] == 0):  
                update3 = close_price * 0.98
                if update3 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update3      



        # if c.position_now > 0:
        #     if (high[c.i-1,c.col] >= tp_prices[c.i])&(tp_prices[c.i] != np.nan):
        #         value = vbt.portfolio.nb.order_nb(
        #             size=-c.position_now/2,  # Close position
        #             price=tp_prices[c.i],
        #             size_type=SizeType.Amount,
        #             direction=Direction.LongOnly)
        #         tp_prices[:] = np.nan
        #         sl_prices[:] = entry_price[c.i]
        #         # print('tp hit at index',c.i)
        #         return value
                


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i, c.col] == 100:
            entry_price[:] = close_price
            tp_prices[:] = close_price * 1.10
            order = vbt.portfolio.nb.order_nb(
                size=0.10,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.0,  # No fees
                slippage=0.0,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )
            lowest_low = np.min(low[c.i-5:c.i,c.col]) 
            sl_prices[:] = lowest_low * 0.98
            return order
    


    return vbt.portfolio.enums.NoOrder


# open_ = comb_open[2000:6000]
# high = comb_high[2000:6000]
# low = comb_low[2000:6000]
# close = comb_close_new.iloc[2000:6000]['btc_close']
# entries = df_custom_indiator.to_numpy()

open_ = comb_open
high = comb_high
low = comb_low
close = df_combined_close
entries = df_custom_indiator.to_numpy()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array
# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,  # Pass the SL prices array
    init_cash=100000,  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable

# Optional: Plot equity curve
# pf.plot().show()



Total Return: ma_rsi_ma1_fast  ma_rsi_ma2_slow  ma_rsi_rsi_buy  ma_rsi_rsi_window
50               400              70              14                   0.131615
                                                  7                    0.199645
Name: total_return, dtype: float64

Order Records:


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(50, 400, 70, 14)",2019-03-16 00:00:00+00:00,71.566593,139.7300,0.0,Buy
1,1,"(50, 400, 70, 14)",2019-03-17 00:00:00+00:00,71.566593,138.0330,0.0,Sell
2,2,"(50, 400, 70, 14)",2019-03-29 08:00:00+00:00,70.760575,141.1500,0.0,Buy
3,3,"(50, 400, 70, 14)",2019-04-04 08:00:00+00:00,70.760575,158.6326,0.0,Sell
4,4,"(50, 400, 70, 14)",2019-04-08 00:00:00+00:00,56.215951,179.8700,0.0,Buy
...,...,...,...,...,...,...,...
519,519,"(50, 400, 70, 7)",2024-12-08 08:00:00+00:00,3.015435,3931.6032,0.0,Sell
520,520,"(50, 400, 70, 7)",2024-12-12 00:00:00+00:00,3.055287,3934.9200,0.0,Buy
521,521,"(50, 400, 70, 7)",2024-12-14 12:00:00+00:00,3.055287,3854.8986,0.0,Sell
522,522,"(50, 400, 70, 7)",2024-12-15 20:00:00+00:00,3.030582,3958.9300,0.0,Buy


: 

In [44]:
pf.total_return()

ma_rsi_ma1_fast  ma_rsi_ma2_slow  ma_rsi_rsi_buy  ma_rsi_rsi_window
50               400              70              14                   0.072260
                                                  7                    0.096621
Name: total_return, dtype: float64

In [45]:
pf.total_return().sort_values(ascending=False).head(25)

ma_rsi_ma1_fast  ma_rsi_ma2_slow  ma_rsi_rsi_buy  ma_rsi_rsi_window
50               400              70              7                    0.096621
                                                  14                   0.072260
Name: total_return, dtype: float64

In [46]:
pf.total_return().sort_values(ascending=False).iloc[800:1000]

Series([], Name: total_return, dtype: float64)

In [47]:
pf.total_return().sort_values(ascending=False).tail(20)

ma_rsi_ma1_fast  ma_rsi_ma2_slow  ma_rsi_rsi_buy  ma_rsi_rsi_window
50               400              70              7                    0.096621
                                                  14                   0.072260
Name: total_return, dtype: float64

In [48]:
pf.plot().show()

/var/folders/33/gyg8_bbj4qn5_tvszhbzjxdm0000gn/T/ipykernel_38589/3894044759.py:1: UserWarning:

Subplot 'orders' raised an exception



TypeError: Only one column is allowed. Use indexing or column argument.

In [95]:
# Load the Portfolio later
pf1 = vbt.Portfolio.load('1h_ma_rsi_PF.pkl')
pf1.orders.records_readable


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(50, 400, 70, btc_close)",2019-02-23 17:00:00+00:00,0.978085,4089.624533,0.0,Buy
1,1,"(50, 400, 70, btc_close)",2019-02-24 15:00:00+00:00,0.978085,4014.361727,0.0,Sell
2,2,"(50, 400, 70, btc_close)",2019-04-02 14:00:00+00:00,0.847642,4715.500000,0.0,Buy
3,3,"(50, 400, 70, btc_close)",2019-04-04 19:00:00+00:00,0.847642,4835.908000,0.0,Sell
4,4,"(50, 400, 70, btc_close)",2019-04-06 15:00:00+00:00,0.787316,5082.000000,0.0,Buy
...,...,...,...,...,...,...,...
163,163,"(50, 400, 70, btc_close)",2024-10-21 15:00:00+00:00,0.061774,67130.000000,0.0,Sell
164,164,"(50, 400, 70, btc_close)",2024-11-10 04:00:00+00:00,0.053922,79106.300000,0.0,Buy
165,165,"(50, 400, 70, btc_close)",2024-11-14 15:00:00+00:00,0.053922,88771.340000,0.0,Sell
166,166,"(50, 400, 70, btc_close)",2024-11-20 14:00:00+00:00,0.045306,94610.100000,0.0,Buy


In [96]:

df_sorted = pf.total_return()
df_sorted.vbt.volume(x_level='ma_rsi_ma1_fast', y_level='ma_rsi_ma2_slow', z_level='ma_rsi_rsi_buy')


AttributeError: 'numpy.float64' object has no attribute 'vbt'

In [42]:

df_sorted = pf.total_return()
df_sorted.vbt.heatmap(x_level='ma_rsi_ma1_fast', y_level='ma_rsi_ma2_slow', slider_level='ma_rsi_rsi_buy')

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'ma_rsi_ma1_fast: %{x}<br>ma_rsi_ma2_slow: %{y}<br>value: %{z}<extra></extra>',
              'name': '70',
              'type': 'heatmap',
              'uid': 'a92102c0-4a86-4338-80c3-fec15089dae5',
              'visible': True,
              'x': array([50]),
              'y': array([400]),
              'z': array([[0.07245347]])}],
    'layout': {'height': 555,
               'legend': {'orientation': 'h',
                         